In [1]:
# Load all necessary packages and modules

import os
import scipy.io
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert


In [2]:
# Insert metadata information

subjects=['al','ca','cc','de','fp','gc','gf','gw',
          'h0','hh','jc','jm','jp','mv','rh','rr',
          'ug','wc','wm','zt']

os.chdir(r'C:\Users\jaapv\Desktop\master\VoytekLab')

# dataset
dataset = 'fixation_pwrlaw'
fs = 1000

# subj info
subj = 16
ch = 17

# get the filename
sub_label = subjects[subj] + '_base'
filename = os.path.join(os.getcwd(), dataset, 'data', sub_label)

# load data
dataStruct = sp.io.loadmat(filename)
data = dataStruct['data']
locs = dataStruct['locs']

sig = data[:,ch]

In [4]:
#%% Filtering and circle correlation functions

def butter_bandpass(lowcut, highcut, fs, order=4):
    #lowcut is the lower bound of the frequency that we want to isolate
    #hicut is the upper bound of the frequency that we want to isolate
    #fs is the sampling rate of our data
    nyq = 0.5 * fs #nyquist frequency - see http://www.dspguide.com/ if you want more info
    low = float(lowcut) / nyq
    high = float(highcut) / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(mydata, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, mydata)
    return y

def circCorr(ang,line):
    n = len(ang)
    rxs = sp.stats.pearsonr(line,np.sin(ang))
    rxs = rxs[0]
    rxc = sp.stats.pearsonr(line,np.cos(ang))
    rxc = rxc[0]
    rcs = sp.stats.pearsonr(np.sin(ang),np.cos(ang))
    rcs = rcs[0]
    rho = np.sqrt((rxc**2 + rxs**2 - 2*rxc*rxs*rcs)/(1-rcs**2)) #r
    r_2 = rho**2 #r squared
    pval = 1- sp.stats.chi2.cdf(n*(rho**2),1)
    standard_error = np.sqrt((1-r_2)/(n-2))

    return rho, pval, r_2,standard_error

In [5]:
#%% Select frequency bands for PAC
    
phase_providing_band = [4,8]; #4-8 Hz band
amplitude_providing_band = [80, 125]; #80-125 Hz band


In [6]:
#### Loop through every subj and channel to find which have PAC
#### This will be saved in the output structure PAC_presence

# create output matrix of 20 * 64 (subj * channels)
PAC_presence = np.full((20,64),np.nan)

# for every subject
for subj in range(len(subjects)): 
    
    # get the filename
    sub_label = subjects[subj] + '_base'
    filename = os.path.join(os.getcwd(), dataset, 'data', sub_label)
    
    # load data
    dataStruct = sp.io.loadmat(filename)
    data = dataStruct['data']
    locs = dataStruct['locs']
    
    # for every channel 
    for ch in range(len(locs)):
        
        
    
        #calculating phase of theta of 20 seconds of the signal
        phase_data = butter_bandpass_filter(data[50000:70000,ch], phase_providing_band[0], phase_providing_band[1], round(float(fs)));
        phase_data_hilbert = hilbert(phase_data);
        phase_data_angle = np.angle(phase_data_hilbert);
        
        #calculating amplitude envelope of high gamma of 20 seconds of the signal
        amp_data = butter_bandpass_filter(data[50000:70000,ch], amplitude_providing_band[0], amplitude_providing_band[1], round(float(fs)));
        amp_data_hilbert = hilbert(amp_data);
        amp_data_abs = abs(amp_data_hilbert);

        # calculate PAC using circCorr function on 2 seconds of the data
        PAC_values = circCorr(phase_data_angle[10000:12000], amp_data_abs[10000:12000])
        
        # save whether there is PAC or not in the matrix
        if PAC_values[1] <= 0.05:
            
            PAC_presence[subj, ch] = 1
            
        elif PAC_values[1] > 0.05: 

            PAC_presence[subj, ch] = 0
            
            
    print('another one is done =), this was subj', subj)


another one is done =), this was subj 0
another one is done =), this was subj 1
another one is done =), this was subj 2
another one is done =), this was subj 3


C:\Users\jaapv\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\jaapv\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\jaapv\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\jaapv\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


another one is done =), this was subj 4
another one is done =), this was subj 5
another one is done =), this was subj 6
another one is done =), this was subj 7
another one is done =), this was subj 8
another one is done =), this was subj 9
another one is done =), this was subj 10
another one is done =), this was subj 11
another one is done =), this was subj 12
another one is done =), this was subj 13
another one is done =), this was subj 14
another one is done =), this was subj 15
another one is done =), this was subj 16
another one is done =), this was subj 17
another one is done =), this was subj 18
another one is done =), this was subj 19


In [15]:
# Save output
np.save('PAC_presence_2s_6062.npy', PAC_presence)   
     

In [ ]:
# Calculate in how much channels PAC is detected
(PAC_presence == 1).sum() / ((PAC_presence == 1).sum() + (PAC_presence == 0).sum()) * 100
